<a href="https://colab.research.google.com/github/phxtho/VideoGameGenrePredictor/blob/master/tensorflow/VideoGamesNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade tensorflow==1.12.0

Requirement already up-to-date: tensorflow==1.12.0 in /usr/local/lib/python3.6/dist-packages (1.12.0)


In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import json
from google.colab import files
import urllib

from sklearn.preprocessing import MultiLabelBinarizer

print(tf.__version__)

1.12.0


## Pre-processing

### Read in data

In [0]:
df = pd.read_csv("vgsales.csv")

In [4]:
df.head(5)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


### Clean up data

Shuffle the data set and remove extrea feature columns

In [0]:
# Drop unwanted columns
unwanted_cols = ['Rank','Platform','Publisher', 'Year' ,'NA_Sales','EU_Sales','JP_Sales','Other_Sales','Global_Sales']
df.drop(unwanted_cols, inplace=True, axis=1)

In [0]:
# Shuffle the dataset
df = df.sample(frac=1).reset_index(drop=True)

In [7]:
df.head(5)

,Name,Genre
0,Obscure,Adventure
1,To Heart 2: Dungeon Travelers,Adventure
2,High School Musical 3: Senior Year DANCE!,Misc
3,Wallace & Gromit: Curse of the Were-Rabbit,Adventure
4,Fuel,Racing


### Splitting the data

80% for training 20% for validation


In [0]:
# Split data into training and validation set
train_size = int(len(df) * .8)
train_df = df.iloc[0:train_size]
valid_df = df.iloc[train_size:len(df)]

In [0]:
target_col = 'Genre'
input_col = 'Name'

train_inputs = train_df[input_col]
train_genres = train_df[target_col]

valid_inputs = valid_df[input_col]
valid_genres = valid_df[target_col]

### Formating labels

Convert genre strings to multi-hot vectors

In [10]:
encoder = MultiLabelBinarizer()
encoder.fit_transform(train_genres)
train_encoded = encoder.transform(train_genres)
valid_encoded = encoder.transform(valid_genres)
num_classes = len(encoder.classes_)

print(encoder.classes_)
print(train_encoded[0])

['-' 'A' 'F' 'M' 'P' 'R' 'S' 'a' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'l' 'm' 'n'
 'o' 'p' 'r' 's' 't' 'u' 'v' 'y' 'z']
[0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 1 1 0 0]


### Embedding layer
Using TF Hub universal-sentence-encoder module for pre-trained word embeddings

In [0]:
name_embeddings = hub.text_embedding_column("Name", module_spec="https://tfhub.dev/google/universal-sentence-encoder/2", trainable=False)

## Language Model

### DNNEstimator Model

The first parameter we pass to our DNNEstimator is called a head, and defines the type of labels our model should expect. Since we want our model to output multiple labels, we’ll use multi_label_head here. Then we'll convert our features and labels to numpy arrays and instantiate our Estimator. batch_size and num_epochs are hyperparameters - you should experiment with different values to see what works best on your dataset.

In [0]:
multi_label_head = tf.contrib.estimator.multi_label_head(
    num_classes,
    loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
)

In [13]:
features = {
  "Name": np.array(train_inputs).astype(np.str)
}

labels = np.array(train_encoded).astype(np.int32)

train_input_fn = tf.estimator.inputs.numpy_input_fn(features, labels, shuffle=True, batch_size=32, num_epochs=25)

estimator = tf.contrib.estimator.DNNEstimator(
    head=multi_label_head,
    hidden_units=[64,10],
    feature_columns=[name_embeddings])

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpn_da1wen', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f92703f3048>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpn_da1wen', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f92703f3048>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


### Training

training and evaluation of the model

In [14]:
estimator.train(input_fn=train_input_fn)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


INFO:tensorflow:Calling model_fn.


...

INFO:tensorflow:loss = 0.33081836, step = 10301 (0.773 sec)


INFO:tensorflow:Saving checkpoints for 10374 into /tmp/tmpn_da1wen/model.ckpt.


INFO:tensorflow:Saving checkpoints for 10374 into /tmp/tmpn_da1wen/model.ckpt.


INFO:tensorflow:Loss for final step: 0.36981753.


INFO:tensorflow:Loss for final step: 0.36981753.


In [15]:
# Define our eval input_fn and run eval
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"Name": np.array(valid_inputs).astype(np.str)}, valid_encoded.astype(np.int32), shuffle=False)
estimator.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving dict for global step 10374: auc = 0.882375, auc_precision_recall = 0.7392524, average_loss = 0.354057, global_step = 10374, loss = 0.35410246


INFO:tensorflow:Saving dict for global step 10374: auc = 0.882375, auc_precision_recall = 0.7392524, average_loss = 0.354057, global_step = 10374, loss = 0.35410246


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10374: /tmp/tmpn_da1wen/model.ckpt-10374


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10374: /tmp/tmpn_da1wen/model.ckpt-10374


{'auc': 0.882375,
 'auc_precision_recall': 0.7392524,
 'average_loss': 0.354057,
 'global_step': 10374,
 'loss': 0.35410246}

## Predictions

In [0]:
# Test our model on some raw name data
raw_test = [
    "Genital Jousting", # Action
    "Firewatch", # Adventure
    "Madden", # Sport
]

In [0]:
# Generate predictions
predict_input_fn = tf.estimator.inputs.numpy_input_fn({"Name": np.array(raw_test).astype(np.str)}, shuffle=False)
results = estimator.predict(predict_input_fn)

In [18]:
# Display predictions
index = 0
text_classes = encoder.inverse_transform(encoder.fit_transform(train_genres))
for game_genres in results:
  print(raw_test[index])
  top_2 = game_genres['probabilities'].argsort()[-2:][::-1]
  for genre in top_2:
    text_genre = encoder.classes_[genre]
    print(text_genre + ': ' + str(round(game_genres['probabilities'][genre] * 100, 2)) + '%')
  index += 1
  print('')

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


Genital Jousting
t: 72.12%
i: 61.99%

Firewatch
o: 81.03%
t: 74.51%

Madden
t: 98.48%
o: 95.57%

